In [1547]:
from collections import OrderedDict
from conllu import parse
from enum import Enum
import numpy as np
import json
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import tokenize_uk
import pymorphy2
import stanza
import spacy
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dropout
from keras.utils import to_categorical
from conllu import parse
from gensim.models.word2vec import Word2Vec
from collections import OrderedDict
import progressbar
import tensorflow as tf
import random as python_random

In [1401]:
PATH = '../../../../UD_Ukrainian-IU'

with open(PATH + '/uk_iu-ud-train.conllu') as f:
    train_data = f.read()
    
with open(PATH + '/uk_iu-ud-dev.conllu') as f:
    test_data = f.read()

train_trees = parse(train_data)
test_trees = parse(test_data)

In [1402]:
nlp_uk = spacy.load('/Users/lissm/Work/Dev/NLP/corpora/vectors/ubercorpus_lowercased_tokenized')

## 1. Oracle LSTM

In [1403]:
class Actions(int, Enum):
    SHIFT = 0
    REDUCE = 1
    RIGHT = 2
    LEFT = 3


ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats',
                                        None), ('head', None),  ('deprel', None),
                    ('deps', None), ('misc', None)])

In [6]:
def calculate_as(trees, clf, vect, feature_extractor):
    total, tpu, tpl, full_match = 0, 0, 0, 0
    golden_u, golden_l = None, None
    for tree in trees:
        tree = [t for t in tree if type(t['id']) == int]
        golden_all = [(node['id'], node['head'], node['deprel'])
                      for node in tree]
        golden_u = [(x, y) for x, y, _ in golden_all]

        predicted_all = dep_parse(tree, clf, vect, feature_extractor)
        predicted_u = [(x, y) for x, y, _ in predicted_all]

        total += len(tree)
        tpu += len(set(golden_u).intersection(set(predicted_u)))
        tpl += len(set(golden_all).intersection(set(predicted_all)))

        if set(golden_all) == set(predicted_all):
            full_match += 1

    print('== Attachment score report ==')
    print('Total: ', total)
    print('Match labeled: ', tpl)
    print('LAS: ', round(tpl/total, 2))
    print("Full match:", round(full_match/len(trees), 2))

In [1574]:
def get_w2v_model(trees):
    words = []
    for tree in trees:
        words.append([x['lemma'] for x in tree])
    model = Word2Vec(
        words, size=300, min_count=1, workers=6,
        window=5, alpha=0.030, negative=7, iter=5
    )

    return model


def get_pos_tag_map(trees):
    pos_map = {'ROOT': 0, 'UNK': 1}
    cnt = 1

    for tree in trees:
        for tok in tree:
            if not pos_map.get(tok['upostag']):
                pos_map[tok['upostag']] = cnt
                cnt += 1
    return pos_map
    

def oracle(stack, top_queue, relations):
    top_stack = stack[-1]
    if top_stack and not top_queue:
        return Actions.REDUCE
    elif top_queue['head'] == top_stack['id']:
        return Actions.RIGHT
    elif top_stack['head'] == top_queue['id']:
        return Actions.LEFT
    elif top_stack['id'] in [i[0] for i in relations] and \
        (top_queue['head'] < top_stack['id'] or
         [s for s in stack if s['head'] == top_queue['id']]):
        return Actions.REDUCE
    else:
        return Actions.SHIFT


def get_data_for_tree(tree, pos_map):
    res = []
    stack, buffer, relations = [ROOT], tree[:], []
    top_feats = 0
    
    while buffer or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        buffer[0] if len(buffer) > 0 else None,
                        relations)
        # we need 3 words from stack and buffer to train the LSTM model
        if top_feats < 3 and len(stack) and len(buffer):
            top_stack = stack[-1]
            top_buff = buffer[0]
            res.append((top_stack['lemma'], pos_map[top_stack['upostag'] or pos_map['UNK']],
                        top_buff['lemma'], pos_map[top_buff['upostag'] or pos_map['UNK']],
                        action.value))
            top_feats += 1
        if action == Actions.SHIFT:
            stack.append(buffer.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            rel = (stack[-1]['id'], buffer[0]['id'])
            relations.append(rel)
            stack.pop()
        elif action == Actions.RIGHT:
            rel = (buffer[0]['id'], stack[-1]['id'])
            relations.append(rel)
            stack.append(buffer.pop(0))

    return res


def get_data(trees, v2w_model, pos_map):
    vectors, labels = [], []
    for tree in trees:
        t_vectors, t_labels = [], []
        tree_data = get_data_for_tree(
            [t for t in tree if type(t['id']) == int], pos_map)
        for stack, stack_pos, buff, buff_pos, label in tree_data:
            try:
                stack_vect = v2w_model.wv[stack]
                buff_vect = v2w_model.wv[buff]
                stack_vect_pos = np.append(stack_vect, [stack_pos])
                buff_vect_pos = np.append(buff_vect, [buff_pos])
                vc = np.concatenate((stack_vect_pos, buff_vect_pos))
                vectors.append(vc)
                labels.append(label)
            except Exception as e:
                pass

    return np.dstack(vectors), np.dstack(labels)


def get_predicted_label(labels):
    l = labels[0][0]
    return np.argmax(l)

In [2184]:
def make_model(n_vocab, n_classes, X):
    model = Sequential()
    model.add(LSTM(256, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(LSTM(512, return_sequences=True, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())

    return model


def to_cat(y_int, vocab_size):
    return to_categorical(y_int, num_classes=vocab_size)


def train_model(model, epochs, x, y, n_classes):
    model.fit(x, y, epochs=epochs, verbose=1)
    

def rolling_window(a, window, step_size):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1 - step_size + 1, window)
    strides = a.strides + (a.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [1406]:
np.random.seed(42)
python_random.seed(42)
tf.random.set_seed(42)

In [1407]:
v2w_model = get_w2v_model(train_trees)

2020-05-26 00:39:11 WARNING: under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [1442]:
pos_map = get_pos_tag_map(train_trees)

In [2154]:
train_x, train_y = get_data(train_trees, v2w_model, pos_map)
n_vocab = len(train_x)
n_classes = 4
x = train_x
y = train_y
x = rolling_window(x, 2, 1)
y = rolling_window(y, 2, 1)
y = np.array([to_cat(x, n_classes) for x in y])
x = np.moveaxis(x, 1, 3)
x = np.lib.stride_tricks.as_strided(x, shape=x.shape[1:], strides=x.strides[1:])
y = np.lib.stride_tricks.as_strided(y, shape=y.shape[2:], strides=y.strides[2:])

In [2155]:
model = make_model(n_vocab, n_classes, x)

Model: "sequential_148"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_238 (LSTM)              (None, 2, 256)            879616    
_________________________________________________________________
lstm_239 (LSTM)              (None, 2, 512)            1574912   
_________________________________________________________________
dense_173 (Dense)            (None, 2, 64)             32832     
_________________________________________________________________
dropout_85 (Dropout)         (None, 2, 64)             0         
_________________________________________________________________
dense_174 (Dense)            (None, 2, 4)              260       
Total params: 2,487,620
Trainable params: 2,487,620
Non-trainable params: 0
_________________________________________________________________
None


In [2156]:
train_model(model, 50, x, y, n_classes)

Epoch 1/50
8803/8803 [==============================] - 14s 2ms/step - loss: 0.9426 - accuracy: 0.6304
Epoch 2/50
8803/8803 [==============================] - 14s 2ms/step - loss: 0.8560 - accuracy: 0.6697
Epoch 3/50
8803/8803 [==============================] - 14s 2ms/step - loss: 0.8204 - accuracy: 0.6845
Epoch 4/50
8803/8803 [==============================] - 13s 2ms/step - loss: 0.7934 - accuracy: 0.6954
Epoch 5/50
8803/8803 [==============================] - 13s 1ms/step - loss: 0.7588 - accuracy: 0.7023
Epoch 6/50
8803/8803 [==============================] - 13s 1ms/step - loss: 0.7345 - accuracy: 0.7062
Epoch 7/50
8803/8803 [==============================] - 13s 1ms/step - loss: 0.6953 - accuracy: 0.7193
Epoch 8/50
8803/8803 [==============================] - 14s 2ms/step - loss: 0.6703 - accuracy: 0.7303
Epoch 9/50
8803/8803 [==============================] - 13s 2ms/step - loss: 0.6533 - accuracy: 0.7422
Epoch 10/50
8803/8803 [==============================] - 13s 1ms/step - l

In [2157]:
model.save('oracle.h5')

## 2. FNN model

In [2286]:
def vectorize_token(stack, buff, pos_map):
    vectors = []
    try:
        stack_vect = v2w_model[stack['lemma']]
    except:
        stack_vect = nlp_uk('unk')[0].vector
    try:
        buff_vect = v2w_model[buff['lemma']]
    except:
        buff_vect = nlp_uk('unk')[0].vector
    stack_pos = pos_map[stack['upostag']] or pos_map['UNK']
    buff_pos = pos_map[buff['upostag']] or pos_map['UNK']
    stack_vect_pos = np.append(stack_vect, [stack_pos])
    buff_vect_pos = np.append(buff_vect, [buff_pos])
    return np.concatenate((stack_vect_pos, buff_vect_pos))


def rolling_window_single(a, window, step_size):
    shape = a.shape[:-1] + (a.shape[-1] - window + 2 - step_size + 1, window)
    strides = a.strides + (a.strides[-1] * step_size,)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


def get_predicted_action(predicted):
    mx = np.amax(predicted)
    result = np.where(predicted == mx)

    return result[2][0]


def get_deps_for_tree(tree, oracle, pos_map, timesteps):
    stack, buffer, relations = [ROOT], tree[:], []
    all_vectors = []

    while buffer or stack:
        if stack and not buffer:
            stack.pop()
        else:
            vects = vectorize_token(stack[-1], buffer[0], pos_map)
            all_vectors.append(vects)
            
            '''
            Якщо чесно, я витратила 100500 часу, але так і не розібралась, як воно має працювати :(
            Особливо не зрозуміло, як працювати з таймстепами. Модель натренувати якось вдалось
            (як мінімум воно не звалилось, хех), але використати predict вже не знаю, як,
            бо не сходяться шейпи. Власне в результаті мені предіктиться в першій же ітерації left, і все розвалюється
            Але в мене вже закінчились час, сили, натхнення та сенс продовжувати воювати з цією бідою,
            бо, здається, я з самого початку щось робила кардинально невірно...
            '''
            rep = np.repeat(vects[np.newaxis,:], timesteps)
            rr = rolling_window_single(vects[np.newaxis,:], 2, 1)
            vectors = rr.reshape(1, timesteps, 602)
   
            predicted = oracle.predict(vectors)
            action = get_predicted_action(predicted)
            print('Action:', action)
            
            if action == Actions.SHIFT:
                stack.append(buffer.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                rel = (stack[-1]['id'], buffer[0]['id'])
                relations.append(rel)
                stack.pop()
            elif action == Actions.RIGHT:
                rel = (buffer[0]['id'], stack[-1]['id'])
                relations.append(rel)
                stack.append(buffer.pop(0))

    return all_vectors, relations


def get_data_for_fnn(trees, oracle, pos_map, timesteps):
    vectors, labels = [], []
    for tree in trees:
        t_vectors, t_labels = [], []
        valid_tree = [t for t in tree if type(t['id']) == int]
        t_vectors, t_deps = get_deps_for_tree(valid_tree, oracle, pos_map, timesteps)

        vectors.append(t_vectors)
        labels.append(t_deps)

    return np.dstack(vectors), np.dstack(labels)


def get_fnn_model(X):
    model = Sequential()
#     model.add(Dense(768, input_dim=(X.shape[1]), init="uniform",
#         activation="relu"))
    model.add(Dense(768, init="uniform", activation="relu"))
    model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
    model.add(Dense(2))
    model.add(Activation("softmax"))
    model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())

    return model

In [2287]:
knn_x = get_data_for_fnn(train_trees, model, pos_map, 2)

Action: 3


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


IndexError: list index out of range

## 3. Курсовий проект - працюю...